In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# # this works successfully. Test with channel pipette only. No tip.

# # ── imports ──────────────────────────────────────────────
# from pylabrobot.liquid_handling import LiquidHandler
# from pylabrobot.liquid_handling.backends import STARBackend
# from pylabrobot.resources.hamilton import STARLetDeck, MFX_CAR_L5_base
# from pylabrobot.resources.hamilton.mfx_modules import MFX_DWP_module_188042
# from pylabrobot.resources.hamilton.mfx_modules import MFX_DWP_rackbased_module

# # ── build LH + deck ──────────────────────────────────────
# backend = STARBackend()
# lh      = LiquidHandler(backend=backend, deck=STARLetDeck())

# # ── module → carrier → deck ─────────────────────────────
# dwp_mod  = MFX_DWP_module_188042("dwp_mod_1")
# other_mod = MFX_DWP_rackbased_module("dwp_mod_2")
# flex_car = MFX_CAR_L5_base("flex_car_1", modules={0: dwp_mod, 1: other_mod})

# lh.deck.assign_child_resource(flex_car, rails=13)

# # ── initialise hardware (run Autoload!) ─────────────────
# await lh.setup(skip_autoload=True)                     

# # ── park channel 0 on the pedestal centre ───────────────
# await lh.backend.prepare_for_manual_channel_operation(0)

# # centre of the module in deck coordinates
# mod_origin = dwp_mod.get_absolute_location()
# center_x   = mod_origin.x + dwp_mod.get_absolute_size_x() / 2
# center_y   = mod_origin.y + dwp_mod.get_absolute_size_y() / 2
# center_z   = mod_origin.z + dwp_mod.get_absolute_size_z()   

# print (center_x, center_y, center_z)

# # 2. traverse in the XY plane
# await lh.backend.move_channel_x(0, center_x)
# await lh.backend.move_channel_y(0, center_y)

# # # 3. descend to the pedestal surface
# await lh.backend.move_channel_z(0, center_z) # z = 184.745

In [ ]:
# ── imports ──────────────────────────────────────────────
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import STARBackend
from pylabrobot.resources.hamilton import (
    STARLetDeck, MFX_CAR_L5_base, TIP_CAR_480_A00               # carrier for tip racks
)
from pylabrobot.resources.hamilton.mfx_modules import (
    MFX_DWP_module_188042, MFX_DWP_rackbased_module
)
from pylabrobot.resources import TIP_50ul_w_filter                           # 50 µL filter tip rack class
import asyncio

# ── build LH + deck ──────────────────────────────────────
backend = STARBackend()
lh      = LiquidHandler(backend=backend, deck=STARLetDeck())

# ── module → carrier → deck ─────────────────────────────
dwp_mod   = MFX_DWP_module_188042("dwp_mod_1")
other_mod = MFX_DWP_rackbased_module("dwp_mod_2")
flex_car  = MFX_CAR_L5_base("flex_car_1", modules={0: dwp_mod, 1: other_mod})
lh.deck.assign_child_resource(flex_car, rails=13)               # ← rails for the module carrier

# ── tip carrier with 50 µL tips ─────────────────────────
tip_car = TIP_CAR_480_A00("tip_car")
tip_car[1] = TIP_50ul_w_filter(name="tips_01")
# tip_rack = fifty_ul_tip_with_filter(name="tips_50ul")                       # slot 2 (index 2) on the carrier
# holder.assign_child_resource(tip_rack)            # mount rack into that holder
lh.deck.assign_child_resource(tip_car, rails=1)   # place the whole carrier on the deck
# lh.deck.assign_child_resource(tip_car, rails=1)                 # ← rails for the tip carrier

# ── initialise hardware (run Autoload!) ─────────────────
await lh.setup(skip_autoload=True)

# ── pick up tip from position 2, well A1 ─────────────────
tiprack = lh.deck.get_resource("tips_01")
await lh.pick_up_tips(tiprack["A1"], use_channels=[3])

# # ── manual control on channel 0 ──────────────────────────
await lh.backend.prepare_for_manual_channel_operation(0)

# # centre of the module in deck coordinates
mod_origin = dwp_mod.get_absolute_location()
center_x   = mod_origin.x + dwp_mod.get_absolute_size_x() / 2
center_y   = mod_origin.y + dwp_mod.get_absolute_size_y() / 2
center_z   = mod_origin.z + dwp_mod.get_absolute_size_z()

print(center_x, center_y, center_z)

# # 1. traverse in the XY plane
# await lh.backend.move_channel_x(0, center_x)
# await lh.backend.move_channel_y(0, center_y)

# # 2. descend to the pedestal surface
# await lh.backend.move_channel_z(0, center_z)

# print ("X coord:", center_x, "Y coord:", center_y, "Z coord:", center_z)

# # 3. pause for visual inspection
# await asyncio.sleep(10)

# # 4. lift clear of the module
# await lh.backend.move_channel_z(0, center_z + 30)               # raise 30 mm (adjust if needed)

# # 5. move back above the original tip position
# tip_loc = tip_pos.get_absolute_location()
# await lh.backend.move_channel_x(0, tip_loc.x)
# await lh.backend.move_channel_y(0, tip_loc.y)
# await lh.backend.move_channel_z(0, tip_loc.z + 10)              # hover 10 mm above rack

# # 6. return tip to rack
# await lh.drop_tips([tip_pos], use_channels=[0])


In [ ]:
    # # ── pick up tip from position 2, well A1 ─────────────────
    # tip_spot = tip_rack["A1"][0]                                      # choose the exact well you want
    # await lh.pick_up_tips([tip_spot], use_channels=[0])

    # # ── manual control on channel 0 ──────────────────────────
    # await lh.backend.prepare_for_manual_channel_operation(0)

    # # centre of the module in deck coordinates
    # mod_origin = dwp_mod.get_absolute_location()
    # center_x   = mod_origin.x + dwp_mod.get_absolute_size_x() / 2
    # center_y   = mod_origin.y + dwp_mod.get_absolute_size_y() / 2
    # center_z   = mod_origin.z + dwp_mod.get_absolute_size_z()

    # # 1. traverse in the XY plane
    # await lh.backend.move_channel_x(0, center_x)
    # await lh.backend.move_channel_y(0, center_y)

    # # 2. descend to the pedestal surface
    # await lh.backend.move_channel_z(0, center_z)

    # print ("X coord:", center_x, "Y coord:", center_y, "Z coord:", center_z)


In [ ]:
# await lh.stop()